# Project 5: Car Crashes in Montgomery County, Maryland

## Problem Statement

Can features such as speed limit, time of day, surface condition, and time of day be used to classify whether a driver is at fault for an automobile crash? 

### Import Libraries

- Import libraries

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)

### Functions

In [83]:
def categorize(df, feature):
    a_list = df[feature].unique().tolist()
    if len(a_list) == 2:
        a_list.sort()
        
    a_dict = {}
    for i in range(0, len(a_list)):
        a_dict[a_list[i]] = i
    
    df[feature] = df[feature].map(a_dict)
    
    return df

### Import data

In [84]:
crash_df = pd.read_csv('./data/crash_reporting_drivers_data.csv', low_memory=False)

In [85]:
crash_df.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Driver At Fault,Injury Severity,Circumstance,Driver Distracted By,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,DD5620004G,190046109,Rockville Police Departme,Property Damage Crash,09/26/2019 07:20:00 AM,NaN,NaN,NaN,NaN,PARKING LOT,NaN,NaN,SINGLE VEHICLE,CLEAR,NaN,DAYLIGHT,NaN,NONE DETECTED,NaN,6558DDB6-C713-4B50-9CDD-4A265FE507B9,Yes,NO APPARENT INJURY,NaN,NOT DISTRACTED,MD,940390C5-2D67-4B95-BCE3-FDF38AA42ECD,SUPERFICIAL,TWELVE OCLOCK,TWELVE OCLOCK,SCHOOL BUS,OTHER,South,South,15,No,No,2017,THOMAS,BUS,NO MISUSE,39.103518,-77.157669,"(39.10351817, -77.15766933)"
1,MCP29620057,200023865,Montgomery County Police,Property Damage Crash,06/18/2020 02:00:00 AM,County,DAIRYMAID DR,County,METZ DR,NaN,NaN,NaN,OTHER,CLOUDY,DRY,UNKNOWN,NaN,UNKNOWN,NaN,7A8F59F9-9886-4D41-B72C-0043BC462A05,Yes,NO APPARENT INJURY,NaN,UNKNOWN,XX,796D6E97-FC6F-4CB5-BCC0-E77F10D6220D,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Unknown,Unknown,35,No,No,2020,UNK,UNK,UNKNOWN,39.154847,-77.271245,"(39.15484667, -77.271245)"
2,MCP3050004K,200016465,Montgomery County Police,Property Damage Crash,04/19/2020 03:39:00 PM,County,RUSSETT RD,County,ARCTIC AVE,NaN,NaN,NaN,OTHER,CLEAR,DRY,DAYLIGHT,NO CONTROLS,ALCOHOL PRESENT,NaN,193D33B6-4BAB-4A82-A4E7-CE35BA3B28F1,Yes,NO APPARENT INJURY,NaN,UNKNOWN,MD,06DEBB17-3C98-421B-A368-8578A48773B2,DISABLING,TWELVE OCLOCK,FOUR OCLOCK,VAN,ACCELERATING,East,East,25,No,No,2004,DODGE,GRAND CARAVAN,NO MISUSE,39.080062,-77.097845,"(39.08006167, -77.097845)"
3,MCP2641001J,200016526,Montgomery County Police,Injury Crash,04/20/2020 09:15:00 AM,County,ARCHDALE RD,County,GUNNERS BRANCH RD,NaN,NaN,NaN,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,5E259E14-1216-4388-87FB-925035107021,Yes,POSSIBLE INJURY,NaN,UNKNOWN,MD,5146C181-7BFF-41CB-ADB8-8AFECD67FCD1,DISABLING,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,UNKNOWN,North,North,25,No,No,2006,HONDA,CR-V,UNKNOWN,39.175230,-77.241090,"(39.17523, -77.24109)"
4,MCP27100039,200016305,Montgomery County Police,Property Damage Crash,04/17/2020 05:00:00 PM,Municipality,E DIAMOND AVE,Municipality,S SUMMIT AVE,NaN,GAITHERSBURG,NaN,SAME DIR REAR END,NaN,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,0F7ECAF5-441D-44C3-B48F-7B00DCCAD53D,No,NO APPARENT INJURY,NaN,NOT DISTRACTED,MD,828900BA-816A-4464-8DD3-2291E72DD1FC,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,North,North,25,No,No,2011,TOYOTA,4S,NaN,39.140555,-77.193347,"(39.140555, -77.19334667)"


- Check shape

In [86]:
crash_df.shape

(133206, 43)

### Data Cleaning

- Check for nulls

In [87]:
crash_df.isnull().sum()

Report Number                          0
Local Case Number                      0
Agency Name                            0
ACRS Report Type                       0
Crash Date/Time                        0
Route Type                         13007
Road Name                          12319
Cross-Street Type                  13018
Cross-Street Name                  12324
Off-Road Description              120888
Municipality                      118553
Related Non-Motorist              129063
Collision Type                       439
Weather                            10479
Surface Condition                  15697
Light                               1122
Traffic Control                    20692
Driver Substance Abuse             24277
Non-Motorist Substance Abuse      129925
Person ID                              0
Driver At Fault                        0
Injury Severity                        0
Circumstance                      108076
Driver Distracted By                   0
Drivers License 

#### Reasoning for dropping columns with <span style="background-color:#ffff00;">null count over 100,000</span>

- `Circumstances`: This column has <span style="background-color:#ffff00;">108076 null values, accounting for 81% of the records</span>.  We cannot simply drop the rows associated with this column.  Since `Surface Conditions` and `Weather` essentially capture the same data _(e.g., DRY, WET, DIRT, SLUSH, OIL, etc)_, we can safely drop this column.

- `Off-Road Description`: off-road description doesn't offer much information, because crashed happed on-road.
- `Municipality`: Municipality doesn't influence whether driver was/was not at fault.
- `Related Non-Motorist`: This variable could offer some insight, but not enough do delete all row where its value is missing.
- `Non-Motorist Substance Abuse`: This variable could offer some insight, but not enough do delete all row where its value is missing.

In [88]:
crash_df.drop(columns = [
                        'Circumstance', 
                        'Off-Road Description', 
                        'Municipality', 
                        'Related Non-Motorist', 
                        'Non-Motorist Substance Abuse'
                        ],
            inplace = True)

#### Reasoning for dropping other columns <span style="background-color:#ffff00;">nulls</span> and non-nulls

- `Report Number`
- `Local Case Number`
- `Agency Name`
- `ACRS Report Type`

These columns contain administrative data that contributes nothting to intended analysis.

In [89]:
crash_df.drop(columns = [
                        'Report Number', 
                        'Local Case Number', 
                        'Agency Name', 
                        'ACRS Report Type'
                        ],
            inplace = True)

- `Route Type`
- `Road Name`
- `Cross-Street Type`
- `Cross-Street Name`
- `Vehicle Continuing Dir`
- `Vehicle Going Dir`
- `Drivers License State`

These features have a high number of null, but, yet, contributes nothing to whether a driver is at fault.

In [90]:
crash_df.drop(columns = [
                        'Route Type', 
                        'Road Name', 
                        'Cross-Street Type',
                        'Cross-Street Name',
                        'Vehicle Continuing Dir',
                        'Vehicle Going Dir',
                        'Drivers License State'
                        ],
            inplace = True)

- `Equipment Problems`

```crash_df['Equipment Problems'].value_counts(normalize=True)``` show that **70%** of values are `NO MISUSE` (including NaN), and **89%** (when NaN is excluded).  Further, the other categories (AIR BAG FAILED, BELTS MISUSED, etc) doesn't determine if driver is/is not at fault.

In [91]:
crash_df.drop(columns = ['Equipment Problems'], inplace = True)

- `Person ID`: does not contribute in determining fault
- `Vehicle ID`: does not contribute in determining fault
- `Driverless Vehicle`: No: 132611, Unknown: 525
- `Vehicle Year`: does not contribute in determining fault
- `Vehicle Make`: `Vehicle Body Type` is more general information
- `Vehicle Model`: `Vehicle Body Type` is more general information
- `Latitude`: part of `Location` feature
- `Longitude`: part of `Location` feature
- `Location`: dummifying will yield high number of features
- `Injury Severity`: does not contribute in determining fault
- `Vehicle Damage Extent`: does not contribute in determining fault

In [92]:
crash_df.drop(columns = [
                        'Person ID',
                        'Vehicle ID',
                        'Driverless Vehicle',
                        'Vehicle Year',
                        'Vehicle Make',
                        'Vehicle Model',
                        'Latitude',
                        'Longitude',
                        'Location',
                        'Injury Severity',
                        'Vehicle Damage Extent'],
            inplace = True)

In [93]:
crash_df.isnull().sum()

Crash Date/Time                       0
Collision Type                      439
Weather                           10479
Surface Condition                 15697
Light                              1122
Traffic Control                   20692
Driver Substance Abuse            24277
Driver At Fault                       0
Driver Distracted By                  0
Vehicle First Impact Location       156
Vehicle Second Impact Location      256
Vehicle Body Type                  1962
Vehicle Movement                    276
Speed Limit                           0
Parked Vehicle                        0
dtype: int64

- `Weather` and `Surface Condition`

Based on common sense, these two features are highly correlated.  `Surface Condition` is more germane. So, we should drop `Weather`.

In [94]:
crash_df.drop(columns = ['Weather'], inplace = True)

#### Categorize `Crash Date/Time`

There may be a relationship between fault and time of day crash occurred.  Later, if this is not the case, then we can delete this feature.

- Convert 'Crash Date/Time' to type datetime

In [95]:
crash_df['Crash Date/Time'] = pd.to_datetime(crash_df['Crash Date/Time'])

- Categorize 'Crash Date/Time'

In [96]:
# Idea borrowed from https://stackoverflow.com/a/59577864
crash_df['Crash Time of Day'] = (crash_df['Crash Date/Time'].dt.hour % 24 + 4) // 4
crash_df['Crash Time of Day'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

- Move new column `Crash Time of Day` to first position

In [97]:
# Idea borrowed from https://www.geeksforgeeks.org/how-to-move-a-column-to-first-position-in-pandas-dataframe/
first_column = crash_df.pop('Crash Time of Day')
crash_df.insert(0, 'Crash Time of Day', first_column)

In [98]:
crash_df.head()

,Crash Time of Day,Crash Date/Time,Collision Type,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Driver At Fault,Driver Distracted By,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Speed Limit,Parked Vehicle
0,Early Morning,2019-09-26 07:20:00,SINGLE VEHICLE,NaN,DAYLIGHT,NaN,NONE DETECTED,Yes,NOT DISTRACTED,TWELVE OCLOCK,TWELVE OCLOCK,SCHOOL BUS,OTHER,15,No
1,Late Night,2020-06-18 02:00:00,OTHER,DRY,UNKNOWN,NaN,UNKNOWN,Yes,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,35,No
2,Noon,2020-04-19 15:39:00,OTHER,DRY,DAYLIGHT,NO CONTROLS,ALCOHOL PRESENT,Yes,UNKNOWN,TWELVE OCLOCK,FOUR OCLOCK,VAN,ACCELERATING,25,No
3,Morning,2020-04-20 09:15:00,OTHER,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,Yes,UNKNOWN,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,UNKNOWN,25,No
4,Evening,2020-04-17 17:00:00,SAME DIR REAR END,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,No,NOT DISTRACTED,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,25,No


- Drop original Crash Date/Time column

In [99]:
crash_df.drop(columns=['Crash Date/Time'], inplace=True)

#### Drop rows containing NaNs
At this point, we can drop any rows containing `NaNs`

In [100]:
crash_df.dropna(inplace=True)

In [101]:
crash_df.shape

(83937, 14)

#### Drop UNKNOWN, OTHER, and blank spaces from dataframe

In [102]:
crash_df = crash_df.replace(to_replace=['UNKNOWN', 'OTHER', 'other', '', ' '], value=np.nan).dropna()

In [103]:
crash_df.shape

(63170, 14)

In [104]:
crash_df.head()

,Crash Time of Day,Collision Type,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Driver At Fault,Driver Distracted By,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Speed Limit,Parked Vehicle
4,Evening,SAME DIR REAR END,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,No,NOT DISTRACTED,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,25,No
5,Early Morning,SAME DIR REAR END,DRY,DAWN,TRAFFIC SIGNAL,NONE DETECTED,No,NOT DISTRACTED,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,40,No
6,Early Morning,SINGLE VEHICLE,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,No,NOT DISTRACTED,ELEVEN OCLOCK,ELEVEN OCLOCK,POLICE VEHICLE/NON EMERGENCY,MOVING CONSTANT SPEED,35,No
12,Noon,SAME DIRECTION SIDESWIPE,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,No,NOT DISTRACTED,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,30,No
17,Early Morning,STRAIGHT MOVEMENT ANGLE,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,No,NOT DISTRACTED,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,40,No


#### Reduce categories for `Driver Substance Abuse`

Presently, there are nine different categories of substance abuse.  For the sake of efficiency, perhaps, we should reduce the categories to just `No` for `NONE DETECTED` and `Yes` for remaining categories.

In [105]:
crash_df['Driver Substance Abuse'].value_counts()

NONE DETECTED                 61249
ALCOHOL PRESENT                1259
ALCOHOL CONTRIBUTED             434
ILLEGAL DRUG PRESENT             85
MEDICATION PRESENT               46
ILLEGAL DRUG CONTRIBUTED         37
COMBINED SUBSTANCE PRESENT       25
MEDICATION CONTRIBUTED           21
COMBINATION CONTRIBUTED          14
Name: Driver Substance Abuse, dtype: int64

In [106]:
crash_df['Driver Substance Abuse'] = crash_df['Driver Substance Abuse'].apply(lambda x: 'No' if x == 'NONE DETECTED' else 'Yes')

In [107]:
crash_df['Driver Substance Abuse'].value_counts()

No     61249
Yes     1921
Name: Driver Substance Abuse, dtype: int64

#### Categorize Feature Values

We should categorize the data by converting string values to integers.

- Make copy of `crash_df` dataframe

In [114]:
crash_cats_df = crash_df.copy(deep=True)

In [115]:
crash_cats_df.shape, crash_df.shape

((63170, 14), (63170, 14))

- Put features into a list to iterate over in next step

In [116]:
features = crash_cats_df.columns.tolist()
# Remove 'Speed Limit,' because its a numeric value. It doesn't need to be categorized.
features.remove('Speed Limit')
print(features)

['Crash Time of Day', 'Collision Type', 'Surface Condition', 'Light', 'Traffic Control', 'Driver Substance Abuse', 'Driver At Fault', 'Driver Distracted By', 'Vehicle First Impact Location', 'Vehicle Second Impact Location', 'Vehicle Body Type', 'Vehicle Movement', 'Parked Vehicle']


In [117]:
for feature in features:
    categorize(crash_cats_df, feature)

In [118]:
crash_cats_df.head()

,Crash Time of Day,Collision Type,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Driver At Fault,Driver Distracted By,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Speed Limit,Parked Vehicle
4,0,0,0,0,0,0,0,0,0,0,0,0,25,0
5,1,0,0,1,1,0,0,0,0,0,0,0,40,0
6,1,1,0,0,2,0,0,0,1,1,1,1,35,0
12,2,2,0,0,0,0,0,0,1,1,0,1,30,0
17,1,3,0,0,1,0,0,0,2,2,0,1,40,0
